In [ ]:
import os
import main_fte

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
# Configurations for the optimisation.
root_dir = os.path.join("/Users/zico/OneDrive - University of Cape Town/msc/data/cheetah_videos")
out_dir = os.path.join("/Users/zico/OneDrive - University of Cape Town/msc/data/analysed")
# If you are running the code remotely on the Linux i9.
# root_dir = os.path.join("/","data", "zico", "cheetah_videos")
# out_dir = os.path.join("/", "data", "zico", "analysed")
data_path = "kinetic_tests/2009_06_18/trail02/run"
start_frame = 401
end_frame = 600
dlc_thresh = 0.8

In [ ]:
# Run the optimisation.
main_fte.run(root_dir, data_path, start_frame, end_frame, dlc_thresh, sync_offset=[{ "cam": 2, "frame": 259}], out_dir_prefix=out_dir, generate_reprojection_videos=False)

In [ ]:
# Plot the 3D trajectory for visual inspection.
main_fte.plot_cheetah(root_dir, data_path, fte_type="fte", out_dir_prefix=out_dir)

In [ ]:
# Generate metrics.
main_fte.metrics(root_dir, data_path, start_frame, end_frame, dlc_thresh)